In [29]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


In [30]:
%run -i data_generation/simple_linear_regression.py
%run -i data_preparation/v0.py
%run -i model_configuration/v0.py


In [31]:
# %load model_training/v0.py

# Defines number of epochs
n_epochs = 1000

for epoch in range(n_epochs):
    # Sets model to TRAIN mode
    model.train()

    # Step 1 - Computes model's predicted output - forward pass
    yhat = model(x_train_tensor)

    # Step 2 - Computes the loss
    loss = loss_fn(yhat, y_train_tensor)

    # Step 3 - Computes gradients for both "b" and "w" parameters
    loss.backward()

    # Step 4 - Updates parameters using gradients and
    # the learning rate
    optimizer.step()
    optimizer.zero_grad()



In [32]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


In [33]:
%%writefile model_configuration/v1.py
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
lr = 0.1
 
torch.manual_seed(42)
# Now we can create a model and send it at once to the device
model = nn.Sequential(nn.Linear(1, 1)).to(device)
 
# Defines an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)
 
# Defines an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')
 
# Creates the train_step function for our model, loss function
# and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

Overwriting model_configuration/v1.py


In [34]:
def make_train_step_fn(model, loss_fn, optimizer):
  # Builds function that performs a step in the train loop
  def perform_train_step_fn(x, y):
    # Sets model to TRAIN mode
    model.train()
 
    # Step 1 - Computes model's predictions - forward pass
    yhat = model(x)
 
    # Step 2 - Computes the loss
    loss = loss_fn(yhat, y)
 
    # Step 3 - Computes gradients for "b" and "w" parameters
    loss.backward()
 
    # Step 4 - Updates parameters using gradients and
    # the learning rate
    optimizer.step()
    optimizer.zero_grad()
 
    # Returns the loss
    return loss.item()
 
  # Returns the function that will be called inside the
  # train loop
  return perform_train_step_fn


In [35]:
%%writefile model_training/v1.py
 
# Defines number of epochs
n_epochs = 1000

losses = []                                              
 
# For each epoch...
for epoch in range(n_epochs):
    # Performs one train step and returns the corresponding loss
    loss = train_step_fn(x_train_tensor, y_train_tensor)
    losses.append(loss)

Overwriting model_training/v1.py


In [36]:
%run -i data_preparation/v0.py
%run -i model_configuration/v1.py
%run -i model_training/v1.py

In [37]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


In [38]:
%%writefile data_preparation/v1.py
 
# Our data was in Numpy arrays, but we need to transform them
# into PyTorch's Tensors
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()
 
# Builds Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor) 
 
# Builds DataLoader
train_loader = DataLoader(                                 
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)



Overwriting data_preparation/v1.py


In [39]:
%%writefile model_training/v2.py
 
# Defines number of epochs
n_epochs = 1000
 
losses = []
 
# For each epoch...
for epoch in range(n_epochs):
    # inner loop
    mini_batch_losses = []                                
    for x_batch, y_batch in train_loader:                 
        # the dataset "lives" in the CPU, so do our mini-batches
        # therefore, we need to send those mini-batches to the
        # device where the model "lives"
        x_batch = x_batch.to(device)                      
        y_batch = y_batch.to(device)                      
 
        # Performs one train step and returns the
        # corresponding loss for this mini-batch
        mini_batch_loss = train_step_fn(x_batch, y_batch) 
        mini_batch_losses.append(mini_batch_loss)         
 
    # Computes average loss over all mini-batches
    # That's the epoch loss
    loss = np.mean(mini_batch_losses)                     
 
    losses.append(loss)


Overwriting model_training/v2.py


In [40]:
%run -i data_preparation/v1.py
%run -i model_configuration/v1.py
%run -i model_training/v2.py


In [41]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9696]])), ('0.bias', tensor([1.0243]))])

In [42]:
# helper function #3
def make_val_step_fn(model, loss_fn):
  # Builds function that performs a step
  # in the validation loop
  def perform_val_step_fn(x, y):
    # Sets model to EVAL mode
    model.eval()   
 
    # Step 1 - Computes our model's predicted output
    # forward pass
    yhat = model(x)
    # Step 2 - Computes the loss
    loss = loss_fn(yhat, y)
    # There is no need to compute Steps 3 and 4,
    # since we don't update parameters during evaluation
    return loss.item()
 
  return perform_val_step_fn

In [43]:
%%writefile model_configuration/v2.py
 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
# Sets learning rate - this is "eta" ~ the "n"-like Greek letter
lr = 0.1
 
torch.manual_seed(42)
# Now we can create a model and send it at once to the device
model = nn.Sequential(nn.Linear(1, 1)).to(device)

# Defines an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Defines an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# Creates the train_step function for our model, loss function
# and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Creates the val_step function for our model and loss function
val_step_fn = make_val_step_fn(model, loss_fn) 


Overwriting model_configuration/v2.py


In [44]:
%run -i model_configuration/v2.py


In [45]:
model.state_dict()

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])

In [46]:
def mini_batch(device, data_loader, step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
 
        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_losses)
    return loss



In [47]:
%%writefile data_preparation/v2.py
 
torch.manual_seed(13)
 
# Builds tensors from Numpy arrays BEFORE split
x_tensor = torch.as_tensor(x).float()                          
y_tensor = torch.as_tensor(y).float()                          
 
# Builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)
 
# Performs the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train, n_val])
 
# Builds a loader of each set
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)
val_loader = DataLoader(dataset=val_data, batch_size=16)


Overwriting data_preparation/v2.py


In [48]:
%run -i data_preparation/v2.py
%run -i model_configuration/v2.py

In [49]:
# Defines number of epochs
n_epochs = 200

losses = []
val_losses = []                                              

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # VALIDATION - no gradients in validation!
    with torch.no_grad():                                     
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)



In [50]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9438]])), ('0.bias', tensor([1.0287]))])

In [51]:
%%writefile model_training/v4.py
 
# Defines number of epochs
n_epochs = 200

losses = []
val_losses = []                                               
 
for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)
 
    # VALIDATION - no gradients in validation!
    with torch.no_grad():                                     
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)


Overwriting model_training/v4.py


In [52]:
%run -i data_preparation/v2.py
%run -i model_configuration/v2.py
%run -i model_training/v4.py

In [53]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9438]])), ('0.bias', tensor([1.0287]))])

In [54]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/test')

dummy_x, dummy_y = next(iter(train_loader))

writer.add_graph(model, dummy_x.to(device))



In [55]:
writer.add_scalars(
    main_tag='loss',
    tag_scalar_dict={'training': loss,
                     'validation': val_loss},
    global_step=epoch
)


In [56]:
%%writefile model_configuration/v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1, 1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction='mean')

train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

val_step_fn = make_val_step_fn(model, loss_fn)

writer = SummaryWriter('runs/simple_linear_regression')

x_dummy, y_dummy = next(iter(train_loader))

writer.add_graph(model, x_dummy.to(device))



Overwriting model_configuration/v3.py


In [58]:
%%writefile model_training/v5.py

n_epochs = 200

losses = []
val_losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # VALIDATION - no gradients in validation!
    with torch.no_grad():
        val_loss = mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)

    writer.add_scalars(
        main_tag='loss',
        tag_scalar_dict={'training': loss,
                         'validation': val_loss},
        global_step=epoch
    )
    writer.close()
    


Writing model_training/v5.py


In [59]:
%run -i data_preparation/v2.py
%run -i model_configuration/v2.py
%run -i model_training/v5.py
